In [3]:
import pandas as pd
import numpy as np
import math
#Part A 1 Generate polynomial and interaction features for a given degree of the polynomial
def polynomialFeatures(X, degree):
    pa = []
    for x in X:
        for i in range(0, len(X)):
            pa = pa + [x * X[i]]
            
    X = X + pa
    
    return X
    
X = [1, 3, 4]
degree = 2

polynomialFeatures(X,degree)


[1, 3, 4, 1, 3, 4, 3, 9, 12, 4, 12, 16]

In [51]:
# Part A 2 Mean Squared Error
def mse(Y_true, Y_pred):
    mse = 0
    for i in range(0, len(Y_true)):
        mse = mse + pow((Y_true[i] - Y_pred[i]),2)
    mse = mse/len(Y_true)
    return mse
    
Y_true = [20, 30, 34, 53, 43]
Y_pred = [23, 28, 41, 59, 39]

mse(Y_true, Y_pred)

22.8

In [58]:
# Part A 3 Compute training and validation errors
def learning_curve(model, X, Y, cv, train_size=1, learning_rate=0.01, epoches=1000, tol=None, regularization=None, lambd=0.0, **kwargs):

    train_scores = []

    for i in range(1, train_size+1):
        x = X[0:train_size*i][:]
        y = Y[0:train_size*i]

        # create model for data
        model.fit(x,y)
        train_scores = train_scores + [model.predict(x,y)]

        #create model for data using k fold
        cv_data = np.array_split(np.column_stack((x,y)), cv)
        count = 0
        for part in cv_data:
            cv_x = pd.DataFrame()
            cv_y = pd.DataFrame()
            cv_x_test = pd.DataFrame()
            cv_y_test = pd.DataFrame()
            for j in range(0, len(cv_data)):
                if(count == j):
                    cv_x_test = part

    return train_scores, []



class Dummy:
    def __init__(self):
        self.X = np.array([])
        self.Y = np.array([])

    def fit(self, X, Y):
        self.X = X
        self.Y = Y
    
    def predict(self, X, Y):
        w = []
        if( len(X) == len(X[0])):
            w = np.matmul(np.linalg.inv(X),np.transpose(Y))
        else:
            w = np.matmul(np.linalg.inv(np.matmul(np.transpose(X), X)),  np.matmul(np.transpose(X),Y))

        y_hat = np.matmul(X, w)

        e = math.sqrt(mse(Y, y_hat))

        return e


   
# testing data
X = np.array([[2,3,4,2,4],[2,4,5,2,5],[3,2,3,5,8],[3,2,4,5,2],[2,5,5,1,8],[2,3,2,3,5],[3,4,2,5,6],[2,3,4,5,6],[2,3,4,4,5],[3,5,4,8,9],[9,9,8,4,5],[8,4,2,8,7],
              [3,9,5,4,5],[3,2,4,6,5],[2,3,4,2,5],[9,3,4,3,4],[9,3,2,3,4],[2,3,4,2,4],[2,3,5,2,5],[2,4,5,3,9],[2,3,4,2,5],[2,5,4,5,9],[2,3,9,8,4],[3,5,6,8,4],[2,5,8,9,5]])

Y = np.array([3,6,3,5,7,4,5,3,4,5,2,3,8,4,5,3,4,5,3,4,3,5,6,5,6]) # training labels

model = Dummy()

learning_curve(model, X, Y, 5, train_size = 5)

[array([[2, 3, 4, 2, 4, 3]]), array([[2, 4, 5, 2, 5, 6]]), array([[3, 2, 3, 5, 8, 3]]), array([[3, 2, 4, 5, 2, 5]]), array([[2, 5, 5, 1, 8, 7]])]
[array([[2, 3, 4, 2, 4, 3],
       [2, 4, 5, 2, 5, 6]]), array([[3, 2, 3, 5, 8, 3],
       [3, 2, 4, 5, 2, 5]]), array([[2, 5, 5, 1, 8, 7],
       [2, 3, 2, 3, 5, 4]]), array([[3, 4, 2, 5, 6, 5],
       [2, 3, 4, 5, 6, 3]]), array([[2, 3, 4, 4, 5, 4],
       [3, 5, 4, 8, 9, 5]])]
[array([[2, 3, 4, 2, 4, 3],
       [2, 4, 5, 2, 5, 6],
       [3, 2, 3, 5, 8, 3]]), array([[3, 2, 4, 5, 2, 5],
       [2, 5, 5, 1, 8, 7],
       [2, 3, 2, 3, 5, 4]]), array([[3, 4, 2, 5, 6, 5],
       [2, 3, 4, 5, 6, 3],
       [2, 3, 4, 4, 5, 4]]), array([[3, 5, 4, 8, 9, 5],
       [9, 9, 8, 4, 5, 2],
       [8, 4, 2, 8, 7, 3]]), array([[3, 9, 5, 4, 5, 8],
       [3, 2, 4, 6, 5, 4],
       [2, 3, 4, 2, 5, 5]])]
[array([[2, 3, 4, 2, 4, 3],
       [2, 4, 5, 2, 5, 6],
       [3, 2, 3, 5, 8, 3],
       [3, 2, 4, 5, 2, 5]]), array([[2, 5, 5, 1, 8, 7],
       [2, 3, 2, 3,

([7.993605777301127e-15,
  0.630694796730642,
  1.3795662759280476,
  1.6028467436632574,
  1.4540137450235846],
 [])